## Collaborative Filtering based Recommender System
Collaborative filtering is probably the most commonly used recommendation algorithm, there are two main types of methods:

- User-based collaborative filtering is based on the user similarity or neighborhood
- Item-based collaborative filtering is based on similarity among items
They both work similarly but for this project we would be using User-Based collaborative filtering.

User-based collaborative filtering looks for users who are similar. This is very similar to the user clustering method where we employed explicit user profiles to calculate user similarity. However, the user profiles may not be available, so how can we determine if two users are similar?

User-item interaction matrix
For most collaborative filtering-based recommender systems, the main dataset format is a 2-D matrix called the user-item interaction matrix. In the matrix, its row is labeled as the user id/index and column labelled to be the item id/index, and the element (i, j) represents the rating of user i to item j.

A user profile can be seen as the user feature vector that mathematically represents a user's learning interests.

#### About Dataset

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/>.

### Building the recommender system

Import necessary libraries

In [ ]:
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd
import pickle

Load movies dataframe `movies.csv`

In [ ]:
movies_df = pd.read_csv('movies.csv')
movies_df.head()

Load user's ratings dataframe `ratings.csv`

In [ ]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head()

Load the ratings dataset into a Surprise Dataset

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

Split the dataset into train and test sets

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

#### Using SVD algorithm

In [ ]:
# Instantiate
SVD_algor = SVD()

# Train the model
SVD_algor.fit(trainset)

# Make predictions for testset
predictions = SVD_algor.test(testset)

# Evaluate the model
svd_rmse = accuracy.rmse(predictions)

#### Using KNNBasic algorithm

In [ ]:
# Instantiate
KNN_algor = KNNBasic(sim_options={'user_based': True})  # User-based collaborative filtering

# Train the model
KNN_algor.fit(trainset)

# Make predictions for testset
predictions = KNN_algor.test(testset)

# Evaluate the model
knn_rmse = accuracy.rmse(predictions)

#### Results

In [ ]:
results = {
    "SVD":[svd_rmse],
    "KNN": [knn_rmse]
          }
# Convert the dictionary to a DataFrame
results_df = pd.DataFrame(results).T
results_df = results_df.rename(columns={0:"RMSE"})
results_df.sort_values(by='RMSE', ascending=False)

From the results, we can see that the KNN algorithm performed best with a RMSE of 0.94225

#### Save the model

In [ ]:
# Save the KNN model to a file
file_name = 'knn_model.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(KNN_algor, file)

### Webscrape image_url and extract url from data in links.csv

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

##### Load links dataframe `links.csv`

In [ ]:
# load 
link_df = pd.read_csv('links.csv')
link_df.head()

Check information of the dataframes

In [ ]:
link_df.info()

In [ ]:
# Create empty columns with null values
link_df['url'] = np.nan
link_df['img_url'] = np.nan


for idx, tmdbId in tqdm(enumerate(link_df['tmdbId']), total = len(link_df['tmdbId'])):
    try:
        # Get url
        url = 'https://www.themoviedb.org/movie/' + str(tmdbId)
        link_df['url'][idx] = url

        # assign the response to a object
        response = requests.get(url)

        # Use BeautifulSoup() to create a BeautifulSoup object from a response text content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find image container
        obj = soup.find('div', 'image_content backdrop').img

        # Get image url
        image_url = 'https://www.themoviedb.org' + obj.get('data-src')

        # Link image url
        link_df['img_url'][idx] = image_url
    except AttributeError:
        image_url = 'https://www.firstcolonyfoundation.org/wp-content/uploads/2022/01/no-photo-available.jpeg'
        link_df['img_url'][idx] = image_url

##### Save link_df

In [ ]:
# Save link_df
link_df.to_csv('link_df.csv', index=False)

###  Make recommendations for a specific user

Import necessary libraries

In [ ]:
import pandas as pd
import pickle

Load movies dataframe movies.csv

In [ ]:
movies_df = pd.read_csv('movies.csv')

Load user's ratings dataframe ratings.csv

In [ ]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head(3)

Load links dataframe ratings.csv

In [ ]:
links_df = pd.read_csv('link_df.csv')
links_df.head(3)

Merge them on the 'movieId' column

In [ ]:
merged_df = pd.merge(movies_df, links_df, on='movieId')
merged_df.head(3)

In [ ]:
# Save merged_df
merged_df.to_csv('merged_df.csv', index=False)

Load the k-NN Model:

In [ ]:
# Load the saved model from the file
file_name = 'knn_model.pkl'
with open(file_name, 'rb') as file:
    loaded_model = pickle.load(file)

Example of using the loaded model for prediction

In [ ]:
user_id = 2
user_movies = ratings_df[ratings_df['userId'] == user_id]['movieId'].unique()
unrated_movies = merged_df[~merged_df['movieId'].isin(user_movies)]

# Get top-10 movie recommendations for the user
k = 10
user_recommendations = loaded_model.get_neighbors(user_id, k=k)
recommended_movies = unrated_movies[unrated_movies['movieId'].isin(user_recommendations)]

recommended_movies

###
#### ***------------------------------------------------THE END!!!----------------------------------------------***
# Author

## [Emuejevoke Eshemitan](https://www.linkedin.com/in/emuejevoke-eshemitan/)
